# Estimación de la regresión logit-ordinal


## Punto de partida

### ¿Cuál es la probabilidad de éxito de un videojuego?

**Depende de la definición de éxito**, digamos que es cuando los videojuegos generan ingresos anuales de entre 5 y 20 millones de USD (M USD)

### ¿Cuáles son los determinantes de los ingresos?

En la industria de los videojuegos, los ingresos dependen de las calificaciones de los videojuegos, los comentarios positivos, el número de seguidores en redes sociales y el tipo de tags que se les asigna en plataformas seguidas por los usuarios.

Implementamos una regresión logit-ordinal para los grupos de ingresos netos en 1 año para los siguientes casos:

1. Menos de 1 M USD
2. Más de 1 M USD y menos de 5 M USD
3. Más de 5 M USD y menos de 20 M USD
4. Más de 20 M USD

In [ ]:
# Librerías necesarias
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.miscmodels.ordinal_model import OrderedModel
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
sns.set(style="whitegrid")

# Omiting WARNINGS
import warnings
warnings.filterwarnings('ignore')

### Datos en la tabla de Datos


In [ ]:
# Cargar datos
datos = pd.read_csv("Datos.csv", encoding = "utf-8")

datos.head(5)

In [ ]:
# 
datos.shape

In [ ]:
#
# Mostrar hasta 300 columnas (puedes cambiar ese número según lo que necesites)
pd.set_option('display.max_columns', 100)

In [ ]:
#
datos.head(5)

### Tabla de frecuencias de la variable de categoría de ingresos

In [ ]:
# Conteo de categorías en 'Cat Rev Year'
datos['Cat Rev Year'].value_counts()

In [ ]:
# Convertir 'Cat Rev Year' en una variable categórica ordenada
from pandas.api.types import CategoricalDtype

orden = ['1. Menos de 1M', '2. Más de 1M y menos de 5M', '3. Más de 5M y menos de 20M', '4. Más de 20M']

cat_tipo = CategoricalDtype(categories = orden, ordered = True)

datos['Cat.Rev.P'] = datos['Cat Rev Year'].astype(cat_tipo)

datos['Cat.Rev.P'].head()

### Create Categorical Vectors of Followers, etc...

In [ ]:
# Crear la variable 'positive_n' a partir de 'positive_year'
datos['positive_n'] = 0
datos.loc[datos['positive_year'] <= 4, 'positive_n'] = 1
datos.loc[(datos['positive_year'] > 4) & (datos['positive_year'] <= 60), 'positive_n'] = 2
datos.loc[datos['positive_year'] > 60, 'positive_n'] = 3
datos['positive_n'] = datos['positive_n'].astype('category')

# Crear la variable 'Followers_n' a partir de 'Followers_year'
datos['Followers_n'] = 0
datos.loc[datos['Followers_year'] <= 50, 'Followers_n'] = 1
datos.loc[(datos['Followers_year'] > 50) & (datos['Followers_year'] <= 500), 'Followers_n'] = 2
datos.loc[datos['Followers_year'] > 500, 'Followers_n'] = 3
datos['Followers_n'] = datos['Followers_n'].astype('category')

In [ ]:
# Conteo de frecuencias
datos['positive_n'].value_counts().sort_index()

In [ ]:
# Conteo de frecuencias
datos['Followers_n'].value_counts().sort_index()

### Tabla de Estadísticas Descriptivas

In [ ]:
# Resumen estadístico
datos[['revenue_year', 'Price', 'year', 'mes', 'Followers_year', 'Reviews_year']].describe()

In [ ]:
# 
datos['Cat.Rev.P'].value_counts(normalize = True)

In [ ]:
# Conteo de frecuencias
datos['positive_n'].value_counts(normalize = True).sort_index()

In [ ]:
# Conteo de frecuencias
datos['Followers_n'].value_counts(normalize = True).sort_index()

### Select subset: "1. Menos de 1M"

In [ ]:
# Filtrar los datos donde 'Cat.Rev.Year' es igual a "1. Menos de 1M"
datos_menos_1m = datos[datos['Cat Rev Year'] == "1. Menos de 1M"]

In [ ]:
# Resumen estadístico
datos_menos_1m[['revenue_year', 'Price', 'year', 'mes', 'Followers_year', 'Reviews_year']].describe()

In [ ]:
# Conteo de frecuencias
datos_menos_1m['positive_n'].value_counts(normalize = True).sort_index()

In [ ]:
# Conteo de frecuencias
datos_menos_1m['Followers_n'].value_counts(normalize = True).sort_index()

### Select subset: "1. Menos de 1M" ~ 5% (1,700)

¿Por qué razón reducimos la muestra? Un desbalance que afecta el resultado.

In [ ]:
#import numpy as np

# Establecer semilla para reproducibilidad
#np.random.seed(1234)

# Seleccionar una muestra aleatoria de 1700 filas sin reemplazo
datos_menos_1m_10p = datos_menos_1m.sample(n = 1700, replace = False, random_state = 1234)

In [ ]:
# Resumen estadístico
datos_menos_1m_10p[['revenue_year', 'Price', 'year', 'mes', 'Followers_year', 'Reviews_year']].describe()

In [ ]:
# Conteo de frecuencias
datos_menos_1m_10p['positive_n'].value_counts(normalize = True).sort_index()

In [ ]:
# Conteo de frecuencias
datos_menos_1m_10p['Followers_n'].value_counts(normalize = True).sort_index()

### Combine subsets: "1. Menos de 1M" ~ 5% (1,700) + "El resto"

In [ ]:
# Filtrar los datos donde 'Cat.Rev.Year' es diferente de "1. Menos de 1M"
datos_otros = datos[datos['Cat Rev Year'] != "1. Menos de 1M"]

# Combinar ambos subconjuntos en uno solo
datos_subset = pd.concat([datos_menos_1m_10p, datos_otros], ignore_index = True)

# Contar ocurrencias por categoría en 'Cat.Rev.Year'
datos_subset['Cat Rev Year'].value_counts().sort_index()

In [ ]:
# Contar ocurrencias por categoría en 'Cat.Rev.Year'
datos_subset['Cat Rev Year'].value_counts(normalize = True).sort_index()

In [ ]:
# Resumen estadístico
datos_subset[['revenue_year', 'Price', 'year', 'mes', 'Followers_year', 'Reviews_year']].describe()

In [ ]:
# Conteo de frecuencias
datos_subset['positive_n'].value_counts(normalize = True).sort_index()

In [ ]:
# Conteo de frecuencias
datos_subset['Followers_n'].value_counts(normalize = True).sort_index()

### Estimación (Running the ordered logit model) 

In [ ]:
# Define dependent and regresors variables
X = datos_subset[[ 'Price', 'Score', 'Followers_n', 'positive_n', 'Adventure', 'Singleplayer', 'Story Rich', 
                   'Horror', '3D', 'Survival', 'Choices Matter', 'Historical', 'Mystery', 'Replay Value', 
                   'Early Access', 'Female Protagonist', 'Lovecraftian', 'Open World', 'Multiplayer', 'Co-op', 
                   'Indie', 'Atmospheric', 'Violent', 'Online Co-Op', 'Third Person', 'Sandbox', 'Strategy', 
                   'Difficult', 'Gore', 'Soundtrack', 'Great Soundtrack', 'Simulation', 'RPG', 
                   'Character Customization', 'Funny', 'Sci-fi', 'Exploration', 'Action', 'Sexual Content' ]]  

y = datos_subset['Cat.Rev.P']

In [ ]:
# https://www.statsmodels.org/stable/generated/statsmodels.miscmodels.ordinal_model.OrderedModel.html
modelo = OrderedModel(y, X, distr = 'logit')

resultados = modelo.fit(method = 'bfgs')

print(resultados.summary())

In [ ]:
#
import joblib  # para guardar el modelo

# Guardar el modelo como archivo
joblib.dump(resultados, 'OLM_01_model.pkl')

### import model ans use in a example data

In [ ]:
# 1. Cargar el modelo desde el archivo
modelo_cargado = joblib.load('OLM_01_model.pkl')

In [ ]:
# 2. Crear el DataFrame de entrada (setup_01)
setup_01 = pd.DataFrame([{
    'Price': 24.99, 'Score': 9, 'Followers_n': 3, 'positive_n': 3,
    'Adventure': 1, 'Singleplayer': 1, 'Story Rich': 0, 'Horror': 0, '3D': 1, 'Survival': 1,
    'Choices Matter': 0, 'Historical': 0, 'Mystery': 0, 'Replay Value': 0, 'Early Access': 1, 
    'Female Protagonist': 0, 'Lovecraftian': 0, 'Open World': 1, 'Multiplayer': 1, 'Co-op': 1,
    'Indie': 1, 'Atmospheric': 0, 'Violent': 0, 'Online Co-Op': 1, 'Third Person': 1, 'Sandbox': 1,
    'Strategy': 0, 'Difficult': 0,  'Gore': 0, 'Soundtrack': 0, 'Great Soundtrack': 0, 'Simulation': 0, 
    'RPG': 1, 'Character Customization': 1, 'Funny': 0, 'Sci-fi': 0, 'Exploration': 0, 'Action': 1,
    'Sexual Content': 0
}]) 

# Convertir setup_01 a array de numpy
exog_array = setup_01.to_numpy()

In [ ]:
# 3. Predecir probabilidades para cada clase
probabilidades = modelo_cargado.model.predict(modelo_cargado.params, exog = exog_array)

In [ ]:
#
probabilidades